In [1]:
from random import randint, random
import collections
import os

import numpy as np
import math
import time
from collections import deque
import torch as T
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import sklearn.neighbors
import sklearn.metrics
import sklearn.mixture
import networkx.algorithms.community as nx_comm
from scipy import stats
import networkx as nx
import matplotlib.pyplot as plt
import PIL
from nvidia.dali.plugin.pytorch import DALIGenericIterator

from imageComponent import *
from remote_read_write import scp_read_wrapper, scp_write_wrapper
scp_Tload = scp_read_wrapper(T.load, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')
scp_Tsave = scp_write_wrapper(T.save, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')


dali_device = 1
device = 'cuda:2'
T.backends.cudnn.benchmark = True
use_amp = False

/public/ly/anaconda3/envs/zyz/lib/python3.10/site-packages/nvidia/dali/backend.py:46: Warning: DALI support for Python 3.10 is experimental and some functionalities may not work.
  deprecation_warning("DALI support for Python 3.10 is experimental and some functionalities "


In [2]:
model = ViTCifar()
root = '/public/home/ly_1112103017/zyz/imageCL/ckpt/CLBadCifarViT2'

model = resnetToCifar(torchvision.models.resnet18())
model.fc = nn.Linear(512, 512)
root = '/public/home/ly_1112103017/zyz/imageCL/ckpt/CLCifarRes18'

In [3]:
dataset = '/public/ly/zyz/imageCL/dataset/cifar10-image2/train'

In [4]:
size = (32, 32)

batch_size = 1000
strength = 1
random_crop_area = [0.2, 1.0]

In [5]:
pipe = DALICLImageFolders(dataset, size, random_crop_area, strength, batch_size = batch_size, num_threads=4, device_id=dali_device)
loader = dali.plugin.pytorch.DALIGenericIterator([pipe], reader_name = 'reader', output_map = ['img1', 'img2', 'label'],
                                                 last_batch_policy = dali.plugin.base_iterator.LastBatchPolicy.PARTIAL)

In [6]:
@T.inference_mode()
def CL_loss_eval(encoder, loader, device, use_amp=True):
    encoder = encoder.to(device, memory_format=T.channels_last).eval()
    scaler = T.cuda.amp.GradScaler(enabled = use_amp)
    loss = []
    for _ in range(1):
        q, k = [], []
        with T.autocast(device_type='cuda', dtype=T.float16, enabled=use_amp):
            for data in loader:
                img1 = data[0]['img1'].to(device, memory_format=T.channels_last)
                img2 = data[0]['img2'].to(device, memory_format=T.channels_last)
                q.append(encoder(img1).cpu())
                k.append(encoder(img2).cpu())
            q = T.cat(q, dim=0).to(device)
            k = T.cat(k, dim=0).to(device)
            #return q, k
            loss.append((contrastive_loss(q, k) + contrastive_loss(k, q)).item())
            #print(loss)
    return sum(loss) / len(loss)

In [ ]:
#if os.path.exists('%s/loss2.pth' %root):
#    loss = T.load('%s/loss2.pth' %root)
#else:
loss = np.zeros(1201)
for i in range(0, 1201, 10):
    if loss[i] != 0 and loss[i] == loss[i]:
        continue
    else:
        print(i)
    t = time.time()
    encoder = model
    encoder.load_state_dict(scp_Tload('%s/%04d.pth' %(root, i)))
    encoder = encoder.to(device, memory_format = T.channels_last)
    loss[i] = CL_loss_eval(encoder, loader, device, use_amp)
    print(time.time() - t)
    scp_Tsave(loss, '%s/loss.pth' %root)
    print(loss[i])

0
54.27974510192871
8.699891090393066
10


encoder = model
encoder.load_state_dict(T.load('%s/%04d.pth' %('/public/home/ly_1112103017/zyz/imageCL/ckpt/BadCifarViT/init', 1256)))
encoder = encoder.to(device, memory_format = T.channels_last)
CL_loss_eval(encoder, loader, device, use_amp)